In [4]:
# --- Config: edit these ---
AGENT_NAME       = "TestUser"          # agent/trader name to filter
DB_PATH          = "accounts.db"     # path to your SQLite DB
CONTAINS         = None              # e.g., "Max turns" to filter messages; or None
START_DATETIME   = None              # e.g., "2025-08-24 00:00:00"; or None
END_DATETIME     = None              # e.g., "2025-08-24 23:59:59"; or None
LIMIT            = 10              # e.g., 2000; or None for no limit
PRINT_TO_SCREEN  = True              # also print results to stdout
WRITE_TEXT_FILE  = False              # write a human-friendly .txt report
WRITE_CSV_FILE   = False              # also write a .csv

# --- Code (no edits needed below) ---
import os
import sqlite3
from pathlib import Path
from datetime import datetime

def query_logs(
    db_path: str,
    agent_name: str,
    contains: str | None = None,
    start: str | None = None,
    end: str | None = None,
    limit: int | None = None,
):
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"DB not found: {db_path}")

    where = ["LOWER(name) = ?"]
    params = [agent_name.lower()]

    if contains:
        where.append("message LIKE ?")
        params.append(f"%{contains}%")
    if start:
        where.append("datetime >= ?")
        params.append(start)
    if end:
        where.append("datetime <= ?")
        params.append(end)

    where_sql = " AND ".join(where)
    limit_sql = f"LIMIT {int(limit)}" if limit is not None else ""

    sql = f"""
        SELECT datetime, type, message
        FROM logs
        WHERE {where_sql}
        ORDER BY datetime DESC
        {limit_sql}
    """

    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.execute(sql, params)
        rows = cur.fetchall()  # list[tuple(datetime, type, message)]
    return rows

def ensure_report_dir():
    out_dir = Path.cwd() / "report"
    out_dir.mkdir(parents=True, exist_ok=True)
    return out_dir

def write_txt(out_path: Path, rows: list[tuple[str,str,str]], agent_name: str):
    with out_path.open("w", encoding="utf-8") as f:
        f.write(f"Logs for agent: {agent_name}\n")
        f.write(f"Generated: {datetime.now().isoformat(timespec='seconds')}\n")
        f.write("-" * 80 + "\n")
        for dt, typ, msg in rows:
            f.write(f"{dt} [{typ}] {msg}\n")

def write_csv(out_path: Path, rows: list[tuple[str,str,str]]):
    import csv
    with out_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["datetime", "type", "message"])
        for r in rows:
            w.writerow(r)

# Run the query
rows = query_logs(
    DB_PATH,
    AGENT_NAME,
    contains=CONTAINS,
    start=START_DATETIME,
    end=END_DATETIME,
    limit=LIMIT,
)

# Print to screen (optional)
if PRINT_TO_SCREEN:
    if not rows:
        print(f"No logs found for '{AGENT_NAME}'.")
    else:
        print(f"--- {len(rows)} rows for {AGENT_NAME} (desc by datetime) ---")
        for dt, typ, msg in rows:
            print(f"{dt} [{typ}] {msg}")

# Write files (optional)
if (WRITE_TEXT_FILE or WRITE_CSV_FILE) and rows:
    out_dir = ensure_report_dir()
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = f"{AGENT_NAME.lower()}_logs_{stamp}"

    if WRITE_TEXT_FILE:
        txt_path = out_dir / f"{base}.txt"
        write_txt(txt_path, rows, AGENT_NAME)
        print(f"Wrote text report: {txt_path}")

    if WRITE_CSV_FILE:
        csv_path = out_dir / f"{base}.csv"
        write_csv(csv_path, rows)
        print(f"Wrote CSV: {csv_path}")


No logs found for 'TestUser'.
